<a href="https://colab.research.google.com/github/mirchandani-mohnish/Musformer/blob/main/ConvAI_Musformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO List
get the dataset
1. ~~mount the drive~~
2. ~~copy or use dataset directly from there ~~
3. ~~make dataset usable ~~
4. ~~make the dataset json file for speechbrain ~~
5. make simple model run - one encoder transformer decoder architecture

## March 21
1. take one recipe from speechbrain recipes and reverse engineer it
  - look at recipes once to do the same
2. or take a small version of demucs
  - research on demucs and see how you can build the same
  - use the same to split against it

In [2]:
import os
import shutil
import random
from google.colab import drive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
from google.colab import auth

# Authenticate with Google Drive
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleAuth().credentials
# drive = GoogleDrive(gauth)


drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
dataset_path = '/content/drive/MyDrive/musdb18/musdb18 (2)/musdb18'
reduced_dataset_path = '/content/drive/MyDrive/musdb18/musdb18'


In [4]:
!pip install musdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 41.2 MB/s eta 0:00:00


In [5]:
import os
import numpy as np
np.float_ = np.float64
import musdb

MUS_DB_PATH = reduced_dataset_path

mus = musdb.DB(root=MUS_DB_PATH)
mus_train = musdb.DB(root=MUS_DB_PATH,subsets="train", split="train")
mus_valid = musdb.DB(root=MUS_DB_PATH,subsets="train", split="valid")
mus_test = musdb.DB(root=MUS_DB_PATH,subsets="test")
print(mus_train[0])
print(mus_test[0])

ANiMAL - Easy Tiger
Arise - Run Run Run


In [6]:
%%capture
# Installing SpeechBrain via pip
BRANCH = 'develop'
!python -m pip install git+https://github.com/speechbrain/speechbrain.git@$BRANCH

# Clone SpeechBrain repository
!git clone https://github.com/speechbrain/speechbrain/

## Dataset


In [7]:
import json
import torchaudio
from speechbrain.utils.data_utils import get_all_files

train_files = []
valid_files = []
test_files = []

def create_json(json_file, mus_obj):

  json_dict = {}
  for i, track in enumerate(mus_obj):
    if i % 10 == 0:
      print(i)

    file_name = track.name
    file_path = track.path
    file_rate = track.rate
    # file_audio = track.audio
    # file_vocal = track.targets['vocals'].audio
    # print(file_name)
    json_dict[file_name] = {
              "file_path": file_path,
              "rate": file_rate
      }
    # print(json_dict[file_name])

    with open(json_file, mode="w") as json_f:
        json.dump(json_dict, json_f, indent=2)

# 80% for training
create_json("train.json", mus_train)
create_json("valid.json", mus_valid)
create_json("test.json", mus_test)

DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover


0
0
0


## DEMUCS V2 - https://github.com/facebookresearch/demucs/tree/v2?tab=readme-ov-file

In [ ]:
%%file hparams_demucs.yaml

# Seed for reproducibility
seed: 1986
__set_seed: !apply:torch.manual_seed [!ref <seed>]

# Folder setup
output_folder: !ref results/VocalSeparation/<seed>
save_folder: !ref <output_folder>/save
train_log: !ref <output_folder>/train_log.txt
data_folder: !ref /content/drive/MyDrive/musdb18/musdb18/

# Dataset paths
train_annotation: train.json
valid_annotation: valid.json
test_annotation: test.json

# Training parameters
number_of_epochs: 100
batch_size: 16
lr: 0.0001
lr_final: 0.00001

# Dataloader options
train_dataloader_opts:
    batch_size: !ref <batch_size>
    shuffle: True

valid_dataloader_opts:
    batch_size: !ref <batch_size>

test_dataloader_opts:
    batch_size: !ref <batch_size>

# Model parameters
input_size: 2  # Input channels (e.g., stereo audio)
hidden_size: 3200  # Hidden size for LSTM/Linear layers
num_layers: 2  # Number of bidirectional LSTM layers
nhead: 8  # Number of attention heads (if using transformers)
dropout: 0.1  # Dropout rate

# Encoder parameters
encoder_layers:
    Encoder1:
        in_channels: 2
        out_channels: 100
    Encoder7:
        in_channels: 100
        out_channels: 200
    Encoder6:
        in_channels: 1600
        out_channels: 3200

# Decoder parameters
decoder_layers:
    Decoder1:
        in_channels: 100
        out_channels: 8  # 4 * 2
    Decoder2:
        in_channels: 200
        out_channels: 100
    Decoder6:
        in_channels: 3200
        out_channels: 1600

# Linear layer
linear_layer:
    in_features: 6400
    out_features: 3200

# Loss function
loss: !new:speechbrain.nnet.loss.si_snr_loss

# Optimizer
opt_class: !name:torch.optim.Adam
    lr: !ref <lr>

# Learning rate scheduler
lr_annealing: !new:speechbrain.nnet.schedulers.LinearScheduler
    initial_value: !ref <lr>
    final_value: !ref <lr_final>
    epoch_count: !ref <number_of_epochs>

# Checkpointer
checkpointer: !new:speechbrain.utils.checkpoints.Checkpointer
    checkpoints_dir: !ref <save_folder>
    recoverables:
        model: !ref <model>
        counter: !ref <epoch_counter>

# Epoch counter
epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter
    limit: !ref <number_of_epochs>

# Modules
modules:
    encoder: !new:torch.nn.ModuleDict
        Encoder1: !new:speechbrain.nnet.CNN.Conv1d
            out_channels: 100
            kernel_size: 8
            stride: 4
            padding: "same"
        Encoder7: !new:speechbrain.nnet.CNN.Conv1d
            out_channels: 200
            kernel_size: 3
            stride: 1
            padding: "same"
        Encoder6: !new:speechbrain.nnet.CNN.Conv1d
            out_channels: 3200
            kernel_size: 8
            stride: 4
            padding: "same"

    decoder: !new:torch.nn.ModuleDict
        Decoder1: !new:speechbrain.nnet.CNN.ConvTranspose1d
            in_channels: 100
            out_channels: 8
            kernel_size: 8
            stride: 4
            padding: "same"
        Decoder2: !new:speechbrain.nnet.CNN.ConvTranspose1d
            in_channels: 200
            out_channels: 100
            kernel_size: 3
            stride: 1
            padding: "same"
        Decoder6: !new:speechbrain.nnet.CNN.ConvTranspose1d
            in_channels: 3200
            out_channels: 1600
            kernel_size: 8
            stride: 4
            padding: "same"

    linear: !new:speechbrain.nnet.linear.Linear
        input_size: 6400
        n_neurons: 3200

    lstm: !new:torch.nn.LSTM
        input_size: 3200
        hidden_size: 3200
        num_layers: 2
        bidirectional: True
        dropout: 0.1

# Model
model: !new:torch.nn.ModuleList
    - - !ref <encoder>
      - !ref <decoder>
      - !ref <linear>
      - !ref <lstm>

In [ ]:
%%file train_demucs.py

#!/usr/bin/env python3
"""Recipe for training a sequence-to-sequence vocal separation system.
The system employs a Transformer encoder, a decoder, and an attention mechanism
between them.

To run this recipe, do the following:
> python train.py hparams/vocal_separation.yaml

With the default hyperparameters, the system employs a Transformer encoder and decoder.
The neural network is trained with the SI-SDR loss objective.
"""

import os
import sys
import torch
import logging
import json
import speechbrain as sb
from hyperpyyaml import load_hyperpyyaml

logger = logging.getLogger(__name__)


# Brain class for vocal separation training
class VocalSeparation(sb.Brain):
    """Class that manages the training loop. See speechbrain.core.Brain."""

    def compute_forward(self, batch, stage):
        """Runs all the computation of the vocal separation model. It returns the
        separated vocals and music signals.

        Arguments
        ---------
        batch : PaddedBatch
            This batch object contains all the relevant tensors for computation.
        stage : sb.Stage
            One of sb.Stage.TRAIN, sb.Stage.VALID, or sb.Stage.TEST.

        Returns
        -------
        vocals : torch.tensor
            Separated vocals signal.
        music : torch.tensor
            Separated music signal.
        """
        # Move the batch to the appropriate device
        batch = batch.to(self.device)

        # Unpack the batch
        mixed_signal, _ = batch.mixed_signal

        # Encoder: Convert waveform to latent representation
        enc_output = self.modules.encoder(mixed_signal)

        # Transformer: Process latent representation
        transformer_output = self.modules.transformer(enc_output)

        # Mask Estimation: Estimate masks for vocals and music
        vocal_mask = torch.sigmoid(self.modules.vocal_mask(transformer_output))
        music_mask = torch.sigmoid(self.modules.music_mask(transformer_output))

        # Apply masks
        vocals = transformer_output * vocal_mask
        music = transformer_output * music_mask

        # Decoder: Convert masked representation back to waveform
        vocals = self.modules.decoder(vocals)
        music = self.modules.decoder(music)

        return vocals, music

    def compute_objectives(self, predictions, batch, stage):
        """Computes the loss given the predicted and targeted outputs.

        Arguments
        ---------
        predictions : tuple
            The output tensor from `compute_forward` (vocals, music).
        batch : PaddedBatch
            This batch object contains all the relevant tensors for computation.
        stage : sb.Stage
            One of sb.Stage.TRAIN, sb.Stage.VALID, or sb.Stage.TEST.

        Returns
        -------
        loss : torch.Tensor
            A one-element tensor used for backpropagating the gradient.
        """
        # Unpack predictions
        vocals_pred, music_pred = predictions

        # Unpack targets
        vocals_target, _ = batch.vocals
        music_target, _ = batch.music

        # Compute SI-SDR loss for vocals and music
        loss_vocals = self.hparams.loss(vocals_pred, vocals_target)
        loss_music = self.hparams.loss(music_pred, music_target)

        # Combine losses
        loss = loss_vocals + loss_music

        return loss

    def on_stage_end(self, stage, stage_loss, epoch):
        """Gets called at the end of an epoch.

        Arguments
        ---------
        stage : sb.Stage
            One of sb.Stage.TRAIN, sb.Stage.VALID, sb.Stage.TEST
        stage_loss : float
            The average loss for all of the data processed in this stage.
        epoch : int
            The currently-starting epoch. This is passed
            `None` during the test stage.
        """
        # Store the train loss until the validation stage.
        if stage == sb.Stage.TRAIN:
            self.train_stats = {"loss": stage_loss}

        # Perform end-of-iteration things, like annealing, logging, etc.
        elif stage == sb.Stage.VALID:
            # Update learning rate
            old_lr, new_lr = self.hparams.lr_annealing(epoch)
            sb.nnet.schedulers.update_learning_rate(self.optimizer, new_lr)

            # The train_logger writes a summary to stdout and to the logfile.
            self.hparams.train_logger.log_stats(
                stats_meta={"epoch": epoch, "lr": old_lr},
                train_stats=self.train_stats,
                valid_stats={"loss": stage_loss},
            )

            # Save the current checkpoint and delete previous checkpoints.
            self.checkpointer.save_and_keep_only(
                meta={"loss": stage_loss}, min_keys=["loss"]
            )

        # We also write statistics about test data to stdout and to the logfile.
        elif stage == sb.Stage.TEST:
            self.hparams.train_logger.log_stats(
                stats_meta={"Epoch loaded": self.hparams.epoch_counter.current},
                test_stats={"loss": stage_loss},
            )


def dataio_prepare(hparams):
    """This function prepares the datasets to be used in the brain class.
    It also defines the data processing pipeline through user-defined functions.

    Arguments
    ---------
    hparams : dict
        This dictionary is loaded from the `train.yaml` file, and it includes
        all the hyperparameters needed for dataset construction and loading.

    Returns
    -------
    datasets : dict
        Dictionary containing "train", "valid", and "test" keys that correspond
        to the DynamicItemDataset objects.
    """
    # Define audio processing pipeline
    @sb.utils.data_pipeline.takes("file_path")
    @sb.utils.data_pipeline.provides("mixed_signal")
    def audio_pipeline(file_path):
        """Processes the mixed audio signal."""
        mixed_signal = sb.dataio.dataio.read_audio(file_path)
        yield mixed_signal

    # Load JSON manifest files
    datasets = {}
    for dataset in ["train", "valid", "test"]:
        with open(hparams[f"{dataset}_annotation"]) as f:
            data = json.load(f)
        datasets[dataset] = sb.dataio.dataset.DynamicItemDataset.from_dict(
            data, dynamic_items=[audio_pipeline], output_keys=["id", "mixed_signal"]
        )

    return datasets


if __name__ == "__main__":
    # Reading command line arguments
    hparams_file, run_opts, overrides = sb.parse_arguments(sys.argv[1:])

    # Load hyperparameters file with command-line overrides
    with open(hparams_file) as fin:
        hparams = load_hyperpyyaml(fin, overrides)

    # Create experiment directory
    sb.create_experiment_directory(
        experiment_directory=hparams["output_folder"],
        hyperparams_to_save=hparams_file,
        overrides=overrides,
    )

    # We can now directly create the datasets for training, valid, and test
    datasets = dataio_prepare(hparams)

    # Trainer initialization
    vocal_separation_brain = VocalSeparation(
        modules=hparams["modules"],
        opt_class=hparams["opt_class"],
        hparams=hparams,
        run_opts=run_opts,
        checkpointer=hparams["checkpointer"],
    )

    # The `fit()` method iterates the training loop, calling the methods
    # necessary to update the parameters of the model. Since all objects
    # with changing state are managed by the Checkpointer, training can be
    # stopped at any point, and will be resumed on next call.
    vocal_separation_brain.fit(
        vocal_separation_brain.hparams.epoch_counter,
        datasets["train"],
        datasets["valid"],
        train_loader_kwargs=hparams["train_dataloader_opts"],
        valid_loader_kwargs=hparams["valid_dataloader_opts"],
    )

    # Load best checkpoint for evaluation
    test_stats = vocal_separation_brain.evaluate(
        test_set=datasets["test"],
        min_key="loss",
        test_loader_kwargs=hparams["test_dataloader_opts"],
    )

In [ ]:
# Delete the output folder to start training from scratch
# (and not from a previous checkpoint).
!rm -rf ./results/DEMUCS/1986
gg
# Run Training
!python train_demucs.py hparams_demucs.yaml --data_folder='/content/drive/MyDrive/musdb18/musdb18/' --device='cuda:0' --number_of_epochs=25

Traceback (most recent call last):
  File "/content/train_demucs.py", line 191, in <module>
    hparams = load_hyperpyyaml(fin, overrides)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/hyperpyyaml/core.py", line 157, in load_hyperpyyaml
    yaml_stream = resolve_references(yaml_stream, overrides, overrides_must_match)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/hyperpyyaml/core.py", line 325, in resolve_references
    _walk_tree_and_resolve("root", preview, preview, file_path)
  File "/usr/local/lib/python3.11/dist-packages/hyperpyyaml/core.py", line 372, in _walk_tree_and_resolve
    current_node[k] = _walk_tree_and_resolve(sub_key, sub_node, tree, file_path)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/hyperpyyaml/core.py", line 366, in _walk_tree_and_resolve
    curr

## Simple Transformer Model

Writing hparams_transformer.yaml


Writing train_transformer.py


## WSJmix


In [9]:
!cp -r /content/speechbrain/recipes/WSJ0Mix/ /content/wsjmix

In [13]:
%%file hparams.yaml

# ################################
# Model: SepFormer for source separation
# https://arxiv.org/abs/2010.13154
# Dataset : Custom dataset
# ################################
#
# Basic parameters
# Seed needs to be set at top of yaml, before objects with parameters are made
#
seed: 1234
__set_seed: !apply:speechbrain.utils.seed_everything [!ref <seed>]

# Data params

# e.g. '/yourpath/wsj0-mix/2speakers'
# end with 2speakers for wsj0-2mix or 3speakers for wsj0-3mix
data_folder: dataset_path

# the path for wsj0/si_tr_s/ folder -- only needed if dynamic mixing is used
# e.g. /yourpath/wsj0-processed/si_tr_s/
# you need to convert the original wsj0 to 8k
# you can do this conversion with the script ../meta/preprocess_dynamic_mixing.py
base_folder_dm: /content/wsjmix/

experiment_name: sepformer-custom
output_folder: /content/results/<experiment_name>/<seed>
train_log: !ref <output_folder>/train_log.txt
save_folder: !ref <output_folder>/save
train_data: train.json
valid_data: valid.json
test_data: test.json
skip_prep: False


# Experiment params
precision: fp32 # bf16, fp16 or fp32
num_spks: 2 # set to 3 for wsj0-3mix
noprogressbar: False
save_audio: True # Save estimated sources on disk
sample_rate: 16000

####################### Training Parameters ####################################
N_epochs: 200
batch_size: 1
lr: 0.00015
clip_grad_norm: 5
loss_upper_lim: 999999  # this is the upper limit for an acceptable loss
# if True, the training sequences are cut to a specified length
limit_training_signal_len: False
# this is the length of sequences if we choose to limit
# the signal length of training sequences
training_signal_len: 32000

# Set it to True to dynamically create mixtures at training time
dynamic_mixing: False

# Parameters for data augmentation
use_wavedrop: False
use_speedperturb: False
use_rand_shift: False
min_shift: -8000
max_shift: 8000

# Speed perturbation
speed_changes: [95, 100, 105]  # List of speed changes for time-stretching

speed_perturb: !new:speechbrain.augment.time_domain.SpeedPerturb
    orig_freq: !ref <sample_rate>
    speeds: !ref <speed_changes>

# Frequency drop: randomly drops a number of frequency bands to zero.
drop_freq_low: 0  # Min frequency band dropout probability
drop_freq_high: 1  # Max frequency band dropout probability
drop_freq_count_low: 1  # Min number of frequency bands to drop
drop_freq_count_high: 3  # Max number of frequency bands to drop
drop_freq_width: 0.05  # Width of frequency bands to drop

drop_freq: !new:speechbrain.augment.time_domain.DropFreq
    drop_freq_low: !ref <drop_freq_low>
    drop_freq_high: !ref <drop_freq_high>
    drop_freq_count_low: !ref <drop_freq_count_low>
    drop_freq_count_high: !ref <drop_freq_count_high>
    drop_freq_width: !ref <drop_freq_width>

# Time drop: randomly drops a number of temporal chunks.
drop_chunk_count_low: 1  # Min number of audio chunks to drop
drop_chunk_count_high: 5  # Max number of audio chunks to drop
drop_chunk_length_low: 1000  # Min length of audio chunks to drop
drop_chunk_length_high: 2000  # Max length of audio chunks to drop

drop_chunk: !new:speechbrain.augment.time_domain.DropChunk
    drop_length_low: !ref <drop_chunk_length_low>
    drop_length_high: !ref <drop_chunk_length_high>
    drop_count_low: !ref <drop_chunk_count_low>
    drop_count_high: !ref <drop_chunk_count_high>

# loss thresholding -- this thresholds the training loss
threshold_byloss: True
threshold: -30

# Encoder parameters
N_encoder_out: 256
out_channels: 256
kernel_size: 16
kernel_stride: 8

# Dataloader options
# Set num_workers: 0 on MacOS due to behavior of the multiprocessing library
dataloader_opts:
    batch_size: !ref <batch_size>
    num_workers: 3


# Specifying the network
Encoder: !new:speechbrain.lobes.models.dual_path.Encoder
    kernel_size: !ref <kernel_size>
    out_channels: !ref <N_encoder_out>


SBtfintra: !new:speechbrain.lobes.models.dual_path.SBTransformerBlock
    num_layers: 4
    d_model: !ref <out_channels>
    nhead: 8
    d_ffn: 1024
    dropout: 0
    use_positional_encoding: True
    norm_before: True

SBtfinter: !new:speechbrain.lobes.models.dual_path.SBTransformerBlock
    num_layers: 4
    d_model: !ref <out_channels>
    nhead: 8
    d_ffn: 1024
    dropout: 0
    use_positional_encoding: True
    norm_before: True

MaskNet: !new:speechbrain.lobes.models.dual_path.Dual_Path_Model
    num_spks: !ref <num_spks>
    in_channels: !ref <N_encoder_out>
    out_channels: !ref <out_channels>
    num_layers: 1
    K: 250
    intra_model: !ref <SBtfintra>
    inter_model: !ref <SBtfinter>
    norm: ln
    linear_layer_after_inter_intra: False
    skip_around_intra: True

Decoder: !new:speechbrain.lobes.models.dual_path.Decoder
    in_channels: !ref <N_encoder_out>
    out_channels: 1
    kernel_size: !ref <kernel_size>
    stride: !ref <kernel_stride>
    bias: False

optimizer: !name:torch.optim.Adam
    lr: !ref <lr>
    weight_decay: 0

loss: !name:speechbrain.nnet.losses.get_si_snr_with_pitwrapper

lr_scheduler: !new:speechbrain.nnet.schedulers.ReduceLROnPlateau
    factor: 0.5
    patience: 2
    dont_halve_until_epoch: 85

epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter
    limit: !ref <N_epochs>

modules:
    encoder: !ref <Encoder>
    decoder: !ref <Decoder>
    masknet: !ref <MaskNet>

checkpointer: !new:speechbrain.utils.checkpoints.Checkpointer
    checkpoints_dir: !ref <save_folder>
    recoverables:
        encoder: !ref <Encoder>
        decoder: !ref <Decoder>
        masknet: !ref <MaskNet>
        counter: !ref <epoch_counter>
        lr_scheduler: !ref <lr_scheduler>

train_logger: !new:speechbrain.utils.train_logger.FileTrainLogger
    save_file: !ref <train_log>


Overwriting hparams.yaml


In [12]:
%%file train.py
#!/usr/bin/env/python3
"""Recipe for training a neural speech separation system on the wsjmix
dataset. The system employs an encoder, a decoder, and a masking network.

To run this recipe, do the following:
> python train.py hparams/sepformer.yaml
> python train.py hparams/dualpath_rnn.yaml
> python train.py hparams/convtasnet.yaml

The experiment file is flexible enough to support different neural
networks. By properly changing the parameter files, you can try
different architectures. The script supports both wsj2mix and
wsj3mix.


Authors
 * Cem Subakan 2020
 * Mirco Ravanelli 2020
 * Samuele Cornell 2020
 * Mirko Bronzi 2020
 * Jianyuan Zhong 2020
"""

import csv
import os
import sys

import numpy as np
import torch
import torch.nn.functional as F
import torchaudio
from hyperpyyaml import load_hyperpyyaml
from tqdm import tqdm

import speechbrain as sb
import speechbrain.nnet.schedulers as schedulers
from speechbrain.core import AMPConfig
from speechbrain.utils.distributed import run_on_main
from speechbrain.utils.logger import get_logger


# Define training procedure
class Separation(sb.Brain):
    def compute_forward(self, mix, targets, stage, noise=None):
        """Forward computations from the mixture to the separated signals."""

        # Unpack lists and put tensors in the right device
        mix, mix_lens = mix
        mix, mix_lens = mix.to(self.device), mix_lens.to(self.device)

        # Convert targets to tensor
        targets = torch.cat(
            [targets[i][0].unsqueeze(-1) for i in range(self.hparams.num_spks)],
            dim=-1,
        ).to(self.device)

        # Add speech distortions
        if stage == sb.Stage.TRAIN:
            with torch.no_grad():
                if self.hparams.use_speedperturb:
                    mix, targets = self.add_speed_perturb(targets, mix_lens)

                    mix = targets.sum(-1)

                if self.hparams.use_wavedrop:
                    mix = self.hparams.drop_chunk(mix, mix_lens)
                    mix = self.hparams.drop_freq(mix)

                if self.hparams.limit_training_signal_len:
                    mix, targets = self.cut_signals(mix, targets)

        # Separation
        mix_w = self.hparams.Encoder(mix)
        est_mask = self.hparams.MaskNet(mix_w)
        mix_w = torch.stack([mix_w] * self.hparams.num_spks)
        sep_h = mix_w * est_mask

        # Decoding
        est_source = torch.cat(
            [
                self.hparams.Decoder(sep_h[i]).unsqueeze(-1)
                for i in range(self.hparams.num_spks)
            ],
            dim=-1,
        )

        # T changed after conv1d in encoder, fix it here
        T_origin = mix.size(1)
        T_est = est_source.size(1)
        if T_origin > T_est:
            est_source = F.pad(est_source, (0, 0, 0, T_origin - T_est))
        else:
            est_source = est_source[:, :T_origin, :]

        return est_source, targets

    def compute_objectives(self, predictions, targets):
        """Computes the sinr loss"""
        return self.hparams.loss(targets, predictions)

    def fit_batch(self, batch):
        """Trains one batch"""
        amp = AMPConfig.from_name(self.precision)
        should_step = (self.step % self.grad_accumulation_factor) == 0

        # Unpacking batch list
        mixture = batch.mix_sig
        targets = [batch.s1_sig, batch.s2_sig]

        if self.hparams.num_spks == 3:
            targets.append(batch.s3_sig)

        with self.no_sync(not should_step):
            if self.use_amp:
                with torch.autocast(
                    dtype=amp.dtype, device_type=torch.device(self.device).type
                ):
                    predictions, targets = self.compute_forward(
                        mixture, targets, sb.Stage.TRAIN
                    )
                    loss = self.compute_objectives(predictions, targets)

                    # hard threshold the easy dataitems
                    if self.hparams.threshold_byloss:
                        th = self.hparams.threshold
                        loss = loss[loss > th]
                        if loss.nelement() > 0:
                            loss = loss.mean()
                    else:
                        loss = loss.mean()

                if (
                    loss.nelement() > 0 and loss < self.hparams.loss_upper_lim
                ):  # the fix for computational problems
                    self.scaler.scale(loss).backward()
                    if self.hparams.clip_grad_norm >= 0:
                        self.scaler.unscale_(self.optimizer)
                        torch.nn.utils.clip_grad_norm_(
                            self.modules.parameters(),
                            self.hparams.clip_grad_norm,
                        )
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                else:
                    self.nonfinite_count += 1
                    logger.info(
                        "infinite loss or empty loss! it happened {} times so far - skipping this batch".format(
                            self.nonfinite_count
                        )
                    )
                    loss.data = torch.tensor(0.0).to(self.device)
            else:
                predictions, targets = self.compute_forward(
                    mixture, targets, sb.Stage.TRAIN
                )
                loss = self.compute_objectives(predictions, targets)

                if self.hparams.threshold_byloss:
                    th = self.hparams.threshold
                    loss = loss[loss > th]
                    if loss.nelement() > 0:
                        loss = loss.mean()
                else:
                    loss = loss.mean()

                if (
                    loss.nelement() > 0 and loss < self.hparams.loss_upper_lim
                ):  # the fix for computational problems
                    loss.backward()
                    if self.hparams.clip_grad_norm >= 0:
                        torch.nn.utils.clip_grad_norm_(
                            self.modules.parameters(),
                            self.hparams.clip_grad_norm,
                        )
                    self.optimizer.step()
                else:
                    self.nonfinite_count += 1
                    logger.info(
                        "infinite loss or empty loss! it happened {} times so far - skipping this batch".format(
                            self.nonfinite_count
                        )
                    )
                    loss.data = torch.tensor(0.0).to(self.device)
        self.optimizer.zero_grad()

        return loss.detach().cpu()

    def evaluate_batch(self, batch, stage):
        """Computations needed for validation/test batches"""
        snt_id = batch.id
        mixture = batch.mix_sig
        targets = [batch.s1_sig, batch.s2_sig]
        if self.hparams.num_spks == 3:
            targets.append(batch.s3_sig)

        with torch.no_grad():
            predictions, targets = self.compute_forward(mixture, targets, stage)
            loss = self.compute_objectives(predictions, targets)

        # Manage audio file saving
        if stage == sb.Stage.TEST and self.hparams.save_audio:
            if hasattr(self.hparams, "n_audio_to_save"):
                if self.hparams.n_audio_to_save > 0:
                    self.save_audio(snt_id[0], mixture, targets, predictions)
                    self.hparams.n_audio_to_save += -1
            else:
                self.save_audio(snt_id[0], mixture, targets, predictions)

        return loss.mean().detach()

    def on_stage_end(self, stage, stage_loss, epoch):
        """Gets called at the end of a epoch."""
        # Compute/store important stats
        stage_stats = {"si-snr": stage_loss}
        if stage == sb.Stage.TRAIN:
            self.train_stats = stage_stats

        # Perform end-of-iteration things, like annealing, logging, etc.
        if stage == sb.Stage.VALID:
            # Learning rate annealing
            if isinstance(
                self.hparams.lr_scheduler, schedulers.ReduceLROnPlateau
            ):
                current_lr, next_lr = self.hparams.lr_scheduler(
                    [self.optimizer], epoch, stage_loss
                )
                schedulers.update_learning_rate(self.optimizer, next_lr)
            else:
                # if we do not use the reducelronplateau, we do not change the lr
                current_lr = self.hparams.optimizer.optim.param_groups[0]["lr"]

            self.hparams.train_logger.log_stats(
                stats_meta={"epoch": epoch, "lr": current_lr},
                train_stats=self.train_stats,
                valid_stats=stage_stats,
            )
            self.checkpointer.save_and_keep_only(
                meta={"si-snr": stage_stats["si-snr"]}, min_keys=["si-snr"]
            )
        elif stage == sb.Stage.TEST:
            self.hparams.train_logger.log_stats(
                stats_meta={"Epoch loaded": self.hparams.epoch_counter.current},
                test_stats=stage_stats,
            )

    def add_speed_perturb(self, targets, targ_lens):
        """Adds speed perturbation and random_shift to the input signals"""

        min_len = -1
        recombine = False

        if self.hparams.use_speedperturb or self.hparams.use_rand_shift:
            # Performing speed change (independently on each source)
            new_targets = []
            recombine = True

            for i in range(targets.shape[-1]):
                new_target = self.hparams.speed_perturb(targets[:, :, i])
                new_targets.append(new_target)
                if i == 0:
                    min_len = new_target.shape[-1]
                else:
                    if new_target.shape[-1] < min_len:
                        min_len = new_target.shape[-1]

            if self.hparams.use_rand_shift:
                # Performing random_shift (independently on each source)
                recombine = True
                for i in range(targets.shape[-1]):
                    rand_shift = torch.randint(
                        self.hparams.min_shift, self.hparams.max_shift, (1,)
                    )
                    new_targets[i] = new_targets[i].to(self.device)
                    new_targets[i] = torch.roll(
                        new_targets[i], shifts=(rand_shift[0],), dims=1
                    )

            # Re-combination
            if recombine:
                if self.hparams.use_speedperturb:
                    targets = torch.zeros(
                        targets.shape[0],
                        min_len,
                        targets.shape[-1],
                        device=targets.device,
                        dtype=torch.float,
                    )
                for i, new_target in enumerate(new_targets):
                    targets[:, :, i] = new_targets[i][:, 0:min_len]

        mix = targets.sum(-1)
        return mix, targets

    def cut_signals(self, mixture, targets):
        """This function selects a random segment of a given length within the mixture.
        The corresponding targets are selected accordingly"""
        randstart = torch.randint(
            0,
            1 + max(0, mixture.shape[1] - self.hparams.training_signal_len),
            (1,),
        ).item()
        targets = targets[
            :, randstart : randstart + self.hparams.training_signal_len, :
        ]
        mixture = mixture[
            :, randstart : randstart + self.hparams.training_signal_len
        ]
        return mixture, targets

    def reset_layer_recursively(self, layer):
        """Reinitializes the parameters of the neural networks"""
        if hasattr(layer, "reset_parameters"):
            layer.reset_parameters()
        for child_layer in layer.modules():
            if layer != child_layer:
                self.reset_layer_recursively(child_layer)

    def save_results(self, test_data):
        """This script computes the SDR and SI-SNR metrics and saves
        them into a csv file"""

        # This package is required for SDR computation
        from mir_eval.separation import bss_eval_sources

        # Create folders where to store audio
        save_file = os.path.join(self.hparams.output_folder, "test_results.csv")

        # Variable init
        all_sdrs = []
        all_sdrs_i = []
        all_sisnrs = []
        all_sisnrs_i = []
        csv_columns = ["snt_id", "sdr", "sdr_i", "si-snr", "si-snr_i"]

        test_loader = sb.dataio.dataloader.make_dataloader(
            test_data, **self.hparams.dataloader_opts
        )

        with open(save_file, "w", newline="", encoding="utf-8") as results_csv:
            writer = csv.DictWriter(results_csv, fieldnames=csv_columns)
            writer.writeheader()

            # Loop over all test sentence
            with tqdm(test_loader, dynamic_ncols=True) as t:
                for i, batch in enumerate(t):
                    # Apply Separation
                    mixture, mix_len = batch.mix_sig
                    snt_id = batch.id
                    targets = [batch.s1_sig, batch.s2_sig]
                    if self.hparams.num_spks == 3:
                        targets.append(batch.s3_sig)

                    with torch.no_grad():
                        predictions, targets = self.compute_forward(
                            batch.mix_sig, targets, sb.Stage.TEST
                        )

                    # Compute SI-SNR
                    sisnr = self.compute_objectives(predictions, targets)

                    # Compute SI-SNR improvement
                    mixture_signal = torch.stack(
                        [mixture] * self.hparams.num_spks, dim=-1
                    )
                    mixture_signal = mixture_signal.to(targets.device)
                    sisnr_baseline = self.compute_objectives(
                        mixture_signal, targets
                    )
                    sisnr_i = sisnr - sisnr_baseline

                    # Compute SDR
                    sdr, _, _, _ = bss_eval_sources(
                        targets[0].t().cpu().numpy(),
                        predictions[0].t().detach().cpu().numpy(),
                    )

                    sdr_baseline, _, _, _ = bss_eval_sources(
                        targets[0].t().cpu().numpy(),
                        mixture_signal[0].t().detach().cpu().numpy(),
                    )

                    sdr_i = sdr.mean() - sdr_baseline.mean()

                    # Saving on a csv file
                    row = {
                        "snt_id": snt_id[0],
                        "sdr": sdr.mean(),
                        "sdr_i": sdr_i,
                        "si-snr": -sisnr.item(),
                        "si-snr_i": -sisnr_i.item(),
                    }
                    writer.writerow(row)

                    # Metric Accumulation
                    all_sdrs.append(sdr.mean())
                    all_sdrs_i.append(sdr_i.mean())
                    all_sisnrs.append(-sisnr.item())
                    all_sisnrs_i.append(-sisnr_i.item())

                row = {
                    "snt_id": "avg",
                    "sdr": np.array(all_sdrs).mean(),
                    "sdr_i": np.array(all_sdrs_i).mean(),
                    "si-snr": np.array(all_sisnrs).mean(),
                    "si-snr_i": np.array(all_sisnrs_i).mean(),
                }
                writer.writerow(row)

        logger.info("Mean SISNR is {}".format(np.array(all_sisnrs).mean()))
        logger.info("Mean SISNRi is {}".format(np.array(all_sisnrs_i).mean()))
        logger.info("Mean SDR is {}".format(np.array(all_sdrs).mean()))
        logger.info("Mean SDRi is {}".format(np.array(all_sdrs_i).mean()))

    def save_audio(self, snt_id, mixture, targets, predictions):
        "saves the test audio (mixture, targets, and estimated sources) on disk"

        # Create output folder
        save_path = os.path.join(self.hparams.save_folder, "audio_results")
        if not os.path.exists(save_path):
            os.mkdir(save_path)

        for ns in range(self.hparams.num_spks):
            # Estimated source
            signal = predictions[0, :, ns]
            signal = signal / signal.abs().max()
            save_file = os.path.join(
                save_path, "item{}_source{}hat.wav".format(snt_id, ns + 1)
            )
            torchaudio.save(
                save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
            )

            # Original source
            signal = targets[0, :, ns]
            signal = signal / signal.abs().max()
            save_file = os.path.join(
                save_path, "item{}_source{}.wav".format(snt_id, ns + 1)
            )
            torchaudio.save(
                save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
            )

        # Mixture
        signal = mixture[0][0, :]
        signal = signal / signal.abs().max()
        save_file = os.path.join(save_path, "item{}_mix.wav".format(snt_id))
        torchaudio.save(
            save_file, signal.unsqueeze(0).cpu(), self.hparams.sample_rate
        )


def dataio_prep(hparams):
    """Creates data processing pipeline"""

    # 1. Define datasets
    train_data = sb.dataio.dataset.DynamicItemDataset.from_csv(
        csv_path=hparams["train_data"],
        replacements={"data_root": hparams["data_folder"]},
    )

    valid_data = sb.dataio.dataset.DynamicItemDataset.from_csv(
        csv_path=hparams["valid_data"],
        replacements={"data_root": hparams["data_folder"]},
    )

    test_data = sb.dataio.dataset.DynamicItemDataset.from_csv(
        csv_path=hparams["test_data"],
        replacements={"data_root": hparams["data_folder"]},
    )

    datasets = [train_data, valid_data, test_data]

    # 2. Provide audio pipelines

    @sb.utils.data_pipeline.takes("mix_wav")
    @sb.utils.data_pipeline.provides("mix_sig")
    def audio_pipeline_mix(mix_wav):
        mix_sig = sb.dataio.dataio.read_audio(mix_wav)
        return mix_sig

    @sb.utils.data_pipeline.takes("s1_wav")
    @sb.utils.data_pipeline.provides("s1_sig")
    def audio_pipeline_s1(s1_wav):
        s1_sig = sb.dataio.dataio.read_audio(s1_wav)
        return s1_sig

    @sb.utils.data_pipeline.takes("s2_wav")
    @sb.utils.data_pipeline.provides("s2_sig")
    def audio_pipeline_s2(s2_wav):
        s2_sig = sb.dataio.dataio.read_audio(s2_wav)
        return s2_sig

    if hparams["num_spks"] == 3:

        @sb.utils.data_pipeline.takes("s3_wav")
        @sb.utils.data_pipeline.provides("s3_sig")
        def audio_pipeline_s3(s3_wav):
            s3_sig = sb.dataio.dataio.read_audio(s3_wav)
            return s3_sig

    sb.dataio.dataset.add_dynamic_item(datasets, audio_pipeline_mix)
    sb.dataio.dataset.add_dynamic_item(datasets, audio_pipeline_s1)
    sb.dataio.dataset.add_dynamic_item(datasets, audio_pipeline_s2)
    if hparams["num_spks"] == 3:
        sb.dataio.dataset.add_dynamic_item(datasets, audio_pipeline_s3)
        sb.dataio.dataset.set_output_keys(
            datasets, ["id", "mix_sig", "s1_sig", "s2_sig", "s3_sig"]
        )
    else:
        sb.dataio.dataset.set_output_keys(
            datasets, ["id", "mix_sig", "s1_sig", "s2_sig"]
        )

    return train_data, valid_data, test_data


if __name__ == "__main__":
    # Load hyperparameters file with command-line overrides
    hparams_file, run_opts, overrides = sb.parse_arguments(sys.argv[1:])
    with open(hparams_file, encoding="utf-8") as fin:
        hparams = load_hyperpyyaml(fin, overrides)

    # Initialize ddp (useful only for multi-GPU DDP training)
    sb.utils.distributed.ddp_init_group(run_opts)

    # Logger info
    logger = get_logger(__name__)

    # Create experiment directory
    sb.create_experiment_directory(
        experiment_directory=hparams["output_folder"],
        hyperparams_to_save=hparams_file,
        overrides=overrides,
    )

    # Update precision to bf16 if the device is CPU and precision is fp16
    if run_opts.get("device") == "cpu" and hparams.get("precision") == "fp16":
        hparams["precision"] = "bf16"

    # Check if wsj0_tr is set with dynamic mixing
    if hparams["dynamic_mixing"] and not os.path.exists(
        hparams["base_folder_dm"]
    ):
        raise ValueError(
            "Please, specify a valid base_folder_dm folder when using dynamic mixing"
        )

    # Data preparation
    # from prepare_data import prepare_wsjmix  # noqa

    # run_on_main(
    #     prepare_wsjmix,
    #     kwargs={
    #         "datapath": hparams["data_folder"],
    #         "savepath": hparams["save_folder"],
    #         "n_spks": hparams["num_spks"],
    #         "skip_prep": hparams["skip_prep"],
    #         "fs": hparams["sample_rate"],
    #     },
    # )

    # Create dataset objects
    # if hparams["dynamic_mixing"]:
    #     from dynamic_mixing import dynamic_mix_data_prep

    #     # if the base_folder for dm is not processed, preprocess them
    #     if "processed" not in hparams["base_folder_dm"]:
    #         # if the processed folder already exists we just use it otherwise we do the preprocessing
    #         if not os.path.exists(
    #             os.path.normpath(hparams["base_folder_dm"]) + "_processed"
    #         ):
    #             from preprocess_dynamic_mixing import resample_folder

    #             print("Resampling the base folder")
    #             run_on_main(
    #                 resample_folder,
    #                 kwargs={
    #                     "input_folder": hparams["base_folder_dm"],
    #                     "output_folder": os.path.normpath(
    #                         hparams["base_folder_dm"]
    #                     )
    #                     + "_processed",
    #                     "fs": hparams["sample_rate"],
    #                     "regex": "**/*.wav",
    #                 },
    #             )
    #             # adjust the base_folder_dm path
    #             hparams["base_folder_dm"] = (
    #                 os.path.normpath(hparams["base_folder_dm"]) + "_processed"
    #             )
    #         else:
    #             print(
    #                 "Using the existing processed folder on the same directory as base_folder_dm"
    #             )
    #             hparams["base_folder_dm"] = (
    #                 os.path.normpath(hparams["base_folder_dm"]) + "_processed"
    #             )

    #     # Collecting the hparams for dynamic batching
    #     dm_hparams = {
    #         "train_data": hparams["train_data"],
    #         "data_folder": hparams["data_folder"],
    #         "base_folder_dm": hparams["base_folder_dm"],
    #         "sample_rate": hparams["sample_rate"],
    #         "num_spks": hparams["num_spks"],
    #         "training_signal_len": hparams["training_signal_len"],
    #         "dataloader_opts": hparams["dataloader_opts"],
    #     }
    #     train_data = dynamic_mix_data_prep(dm_hparams)
    #     _, valid_data, test_data = dataio_prep(hparams)
    # else:
    #     train_data, valid_data, test_data = dataio_prep(hparams)

    # Load pretrained model if pretrained_separator is present in the yaml
    if "pretrained_separator" in hparams:
        run_on_main(hparams["pretrained_separator"].collect_files)
        hparams["pretrained_separator"].load_collected()

    # Brain class initialization
    separator = Separation(
        modules=hparams["modules"],
        opt_class=hparams["optimizer"],
        hparams=hparams,
        run_opts=run_opts,
        checkpointer=hparams["checkpointer"],
    )

    # re-initialize the parameters if we don't use a pretrained model
    if "pretrained_separator" not in hparams:
        for module in separator.modules.values():
            separator.reset_layer_recursively(module)

    # Training
    separator.fit(
        separator.hparams.epoch_counter,
        train_data,
        valid_data,
        train_loader_kwargs=hparams["dataloader_opts"],
        valid_loader_kwargs=hparams["dataloader_opts"],
    )

    # Eval
    separator.evaluate(test_data, min_key="si-snr")
    separator.save_results(test_data)


Writing train.py


# Archive

---
---
---
---
---
---

In [ ]:
#
import kagglehub
import shutil


path = kagglehub.dataset_download("dakshsethi/musdb18")

print("Path: ", path)



destination_path = "/content/musdb"
db_path = "/content/musdb"
# Move the dataset
shutil.move(path, destination_path)
shutil.move(destination_path + '/musdb18 (2)/musdb18', '/content/')
db_path = '/content/musdb18'
print("Dataset moved to:", db_path)


 23%|██▎       | 0.99G/4.37G [00:16<00:55, 65.3MB/s]


KeyboardInterrupt: 

In [ ]:
import gdown

# Folder ID from the Google Drive link
folder_id = "1-9VnggJq4dTGzbWvLHZMWb5X1LYvofOi"

# Download the folder
gdown.download_folder(f"https://drive.google.com/drive/folders/19NaewsdYy8TgRn3AzbKLYiOeVh02GK9c", quiet=False)

print("Download complete!")

Retrieving folder contents


Retrieving folder 19agTXmjp5xB8MOJzxWOXCnVuKMSLhdhJ test
Processing file 19fPvncz6GIowwfFiR8SUYPsZzYuXOl9- Arise - Run Run Run.stem.mp4
Processing file 19gsWaBKuTdtKgXsISQxpnb1wv4iRX_a_ Ben Carrigan - We will Talk About It All Tonight.stem.mp4
Processing file 19gIET2rx8fUBB1aH9I7jYtsYUvsyaIPe BKS - Bulldozer.stem.mp4
Processing file 19k3TPpqbWkqDd6M-MrvLKR2P8_cb1Og7 Secretariat - Over The Top.stem.mp4
Processing file 19dZ7AaK6UG9_vvGXLsvfJv4mgzndMVYO The Long Wait - Dark Horses.stem.mp4
Retrieving folder 19WzbsAvBhC32b_gXHw_F9HU8zn9Pz2DS train
Processing file 19m1LtGLsoliZup5S0I4Scz-1f_-UarS2 ANiMAL - Easy Tiger.stem.mp4
Processing file 19ocHBeBVF_hRA0TtkBqDN2iUQzMikc29 AvaLuna - Waterduct.stem.mp4
Processing file 19n0ML2L3ht993Ubked5RCkuomZ9mRgJK Bill Chudziak - Children Of No-one.stem.mp4
Processing file 19zOIMlIZGXfZs9UpMOEdvhVPn7eK6xC7 Cnoc An Tursa - Bannockburn.stem.mp4
Processing file 1A4uX-vCsJAEDKBt7vQXfykQDHqurT1i_ Leaf - Summerghost.stem.mp4
Processing file 1A74DzKn1Ww6Ki39W

Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=19fPvncz6GIowwfFiR8SUYPsZzYuXOl9-
To: /content/musdb18/test/Arise - Run Run Run.stem.mp4
100%|██████████| 35.3M/35.3M [00:00<00:00, 157MB/s]
Downloading...
From: https://drive.google.com/uc?id=19gsWaBKuTdtKgXsISQxpnb1wv4iRX_a_
To: /content/musdb18/test/Ben Carrigan - We will Talk About It All Tonight.stem.mp4
100%|██████████| 41.0M/41.0M [00:00<00:00, 90.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=19gIET2rx8fUBB1aH9I7jYtsYUvsyaIPe
To: /content/musdb18/test/BKS - Bulldozer.stem.mp4
100%|██████████| 54.2M/54.2M [00:00<00:00, 114MB/s]
Downloading...
From: https://drive.google.com/uc?id=19k3TPpqbWkqDd6M-MrvLKR2P8_cb1Og7
To: /content/musdb18/test/Secretariat - Over The Top.stem.mp4
100%|██████████| 40.4M/40.4M [00:00<00:00, 45.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=19dZ7AaK6UG9_vvGXLsvfJv4mgzndMVYO
To:

Download complete!



Download completed


In [ ]:
!pip install musdb

In [ ]:
!git clone https://github.com/pyenv/pyenv.git ~/.pyenv

# Set the root for pyenv
os.environ['PYENV_ROOT'] = os.path.expanduser("~/.pyenv")
# Prepend pyenv's bin folder to PATH
os.environ['PATH'] = os.environ['PYENV_ROOT'] + '/bin:' + os.environ['PATH']

# Confirm pyenv is callable
!pyenv --version
!apt-get install libffi-dev
!pyenv install 3.8.13
!pyenv global 3.8.13
!pyenv exec pip install spleeter
!pyenv exec spleeter

Cloning into '/root/.pyenv'...
remote: Enumerating objects: 25664, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 25664 (delta 22), reused 19 (delta 15), pack-reused 25622 (from 3)
Receiving objects: 100% (25664/25664), 5.90 MiB | 21.75 MiB/s, done.
Resolving deltas: 100% (17226/17226), done.
pyenv 2.5.4-1-gc579b636
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libffi-dev
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 63.7 kB of archives.
After this operation, 336 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libffi-dev amd64 3.4.2-4 [63.7 kB]
Fetched 63.7 kB in 0s (181 kB/s)
Selecting previously unselected package libffi-dev:amd64.
(Reading database ... 125044 files and directories currently installed.)
Preparing to unpack .../libffi-dev_3.4

In [ ]:
db_path = '/content/musdb18'

In [ ]:
import os
import musdb

MUS_DB_PATH = db_path

mus = musdb.DB(root=MUS_DB_PATH)
mus_train = musdb.DB(root=MUS_DB_PATH,subsets="train", split="train")
mus_valid = musdb.DB(root=MUS_DB_PATH,subsets="train", split="valid")
mus_test = musdb.DB(root=MUS_DB_PATH,subsets="test")
print(mus_train[0])
print(mus_test[0])

AttributeError: `np.float_` was removed in the NumPy 2.0 release. Use `np.float64` instead.

In [ ]:
%%capture
# Installing SpeechBrain via pip
BRANCH = 'develop'
!python -m pip install git+https://github.com/speechbrain/speechbrain.git@$BRANCH

# Clone SpeechBrain repository
!git clone https://github.com/speechbrain/speechbrain/

## Musformer
Custom Architecture to device a vocal classifier


### Step1 : Simple Transformer Based Architecture


In [ ]:
# ############################################################################
# Model: Transformer-based Vocal Separation
# Encoder: Transformer
# Decoder: Transformer
# Input: Mixed audio waveform
# Output: Separated vocals and music waveforms
# Loss: SI-SDR (Scale-Invariant Signal-to-Distortion Ratio)
# Training: MUSDB18 dataset
##############################################################################

# Seed needs to be set at top of yaml, before objects with parameters are instantiated
seed: 1986
__set_seed: !apply:torch.manual_seed [!ref <seed>]

# Folder set up
output_folder: !ref results/VocalSeparation/<seed>
save_folder: !ref <output_folder>/save
train_log: !ref <output_folder>/train_log.txt
test_log: !ref <output_folder>/test_log.txt

# Path where data manifest files are stored
train_annotation: train.csv
valid_annotation: valid.csv
test_annotation: test.csv

# The train logger writes training statistics to a file, as well as stdout.
train_logger: !new:speechbrain.utils.train_logger.FileTrainLogger
    save_file: !ref <train_log>

# Training parameters
number_of_epochs: 100
batch_size: 16
lr: 0.0001
lr_final: 0.00001

# Dataloader options
train_dataloader_opts:
    batch_size: !ref <batch_size>
    shuffle: True

valid_dataloader_opts:
    batch_size: !ref <batch_size>

test_dataloader_opts:
    batch_size: !ref <batch_size>

# Model parameters
sample_rate: 16000  # Audio sample rate
d_model: 256        # Transformer model dimension
nhead: 8            # Number of attention heads
num_encoder_layers: 4  # Number of encoder layers
num_decoder_layers: 4  # Number of decoder layers
dim_feedforward: 1024  # Feedforward layer dimension
dropout: 0.1        # Dropout rate
kernel_size: 3      # Kernel size for convolutional layers

# Encoder: Converts waveform to latent representation
encoder: !new:speechbrain.lobes.models.CNN.Conv1d
    out_channels: !ref <d_model>
    kernel_size: !ref <kernel_size>
    stride: 1
    padding: "same"

# Transformer: Processes latent representation for separation
transformer: !new:torch.nn.Transformer
    d_model: !ref <d_model>
    nhead: !ref <nhead>
    num_encoder_layers: !ref <num_encoder_layers>
    num_decoder_layers: !ref <num_decoder_layers>
    dim_feedforward: !ref <dim_feedforward>
    dropout: !ref <dropout>
    batch_first: True

# Decoder: Converts latent representation back to waveform
decoder: !new:speechbrain.lobes.models.CNN.ConvTranspose1d
    in_channels: !ref <d_model>
    out_channels: 2  # Output channels for vocals and music
    kernel_size: !ref <kernel_size>
    stride: 1
    padding: "same"

# Positional encoding for transformer
pos_enc: !new:speechbrain.lobes.models.transformer.Transformer.PositionalEncoding
    input_size: !ref <d_model>
    max_len: 5000  # Maximum sequence length

# Loss function: SI-SDR (Scale-Invariant Signal-to-Distortion Ratio)
loss: !new:speechbrain.nnet.loss.si_snr_loss

# The first object passed to the Brain class is this "Epoch Counter"
# which is saved by the Checkpointer so that training can be resumed
# if it gets interrupted at any point.
epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter
    limit: !ref <number_of_epochs>

# Objects in "modules" dict will have their parameters moved to the correct
# device, as well as having train()/eval() called on them by the Brain class
modules:
    encoder: !ref <encoder>
    transformer: !ref <transformer>
    decoder: !ref <decoder>
    pos_enc: !ref <pos_enc>

# Gathering all the submodels in a single model object.
model: !new:torch.nn.ModuleList
    - - !ref <encoder>
      - !ref <transformer>
      - !ref <decoder>

# This function manages learning rate annealing over the epochs.
lr_annealing: !new:speechbrain.nnet.schedulers.LinearScheduler
    initial_value: !ref <lr>
    final_value: !ref <lr_final>
    epoch_count: !ref <number_of_epochs>

# This optimizer will be constructed by the Brain class after all parameters
# are moved to the correct device. Then it will be added to the checkpointer.
opt_class: !name:torch.optim.Adam
    lr: !ref <lr>

# This object is used for saving the state of training both so that it
# can be resumed if it gets interrupted, and also so that the best checkpoint
# can be later loaded for evaluation or inference.
checkpointer: !new:speechbrain.utils.checkpoints.Checkpointer
    checkpoints_dir: !ref <save_folder>
    recoverables:
        model: !ref <model>
        counter: !ref <epoch_counter>

In [ ]:
%%file train_transformer.py
#!/usr/bin/env/python3
"""Recipe for training a sequence-to-sequence machine translation system
on "ignotush".
The system employs a Transformer encoder, a decoder, and an attention mechanism
between them.

To run this recipe, do the following:
> python train.py hparams/Transformers.yaml

With the default hyperparameters, the system employs a Transformer encoder and decoder.

The neural network is trained with the negative-log likelihood objective and
characters are used as basic tokens for both english and ignotush.
"""

import os
import sys
import torch
import logging
import speechbrain as sb
from hyperpyyaml import load_hyperpyyaml

logger = logging.getLogger(__name__)


# Brain class for speech recognition training
class Translate(sb.Brain):
    """Class that manages the training loop. See speechbrain.core.Brain."""

    def compute_forward(self, batch, stage):
        """Runs all the computation of the CTC + seq2seq ASR. It returns the
        posterior probabilities of the CTC and seq2seq networks.

        Arguments
        ---------
        batch : PaddedBatch
            This batch object contains all the relevant tensors for computation.
        stage : sb.Stage
            One of sb.Stage.TRAIN, sb.Stage.VALID, or sb.Stage.TEST.

        Returns
        -------
        predictions : torch.tensor
            Log-probabilities predicted by the decoder.

        At validation/test time, it returns the predicted tokens as well.
        """
        # We first move the batch to the appropriate device.
        # Your code here. Aim for 1 line.
        batch = batch.to(self.device)

        # Unpacking ignotush_encoded_chars and english_encoded_chars_bos
        # Your code here. Aim for 1 line.
        enc_ignotush, inp_lens = batch.ignotush_encoded_chars
        enc_english_bos,  out_lens = batch.english_encoded_chars_bos

        # Input embeddings
        # Your code here. Aim for 1 line.
        enc_emb = self.modules.encoder_emb(enc_ignotush)

        # Positional Embeddings
        # Your code here. Aim for 1 line.
        pos_emb_enc = self.modules.pos_emb_enc(enc_emb)

        # Summing up embeddings
        # Your code here. Aim for 1 line.
        enc_emb = pos_emb_enc + enc_emb

        # Decoding embeddings
        # Your code here. Aim for 3 lines.
        dec_emb = self.modules.decoder_emb(enc_english_bos)
        pos_emb_dec = self.modules.pos_emb_dec(dec_emb)
        # Positional Embeddings
        dec_emb = pos_emb_dec + dec_emb

        # Getting target mask (to avoid looking ahead)
        # Your code here. Aim for 1 line.
        tgt_mask = self.hparams.lookahead_mask(enc_english_bos)

        # Getting the source mask (all zeros is fine in this case to allow the
        # network to embed both past and future contect)
        # Your code here. Aim for 1 line.
        src_mask = torch.zeros(enc_ignotush.size(1), enc_ignotush.size(1))

        # Padding masks for source and targets (use padding_mas)
        # Your code here. Aim for 2 lines.
        src_key_padding_mask = self.hparams.padding_mask(enc_ignotush)
        tgt_key_padding_mask = self.hparams.padding_mask(enc_english_bos)

        # Running the Seq2Seq Transformer
        # Your code here. Aim for 1 line.
        decoder_outputs = self.modules.Seq2SeqTransformer(
            src=enc_emb,
            tgt=dec_emb,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            memory_key_padding_mask=src_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask
        )

        # Compute logits
        # Your code here. Aim for 1 line.
        logits = self.modules.seq_lin(decoder_outputs)

        # Apply log softmax
        # Your code here. Aim for 1 line.
        predictions = self.hparams.log_softmax(logits)


        if stage == sb.Stage.TEST:

            # Greedy Decoding
            hyps = predictions.argmax(-1)

            # getting the first index where the prediciton is eos_index
            stop_indexes = (hyps==self.hparams.eos_index).int()
            stop_indexes = stop_indexes.argmax(dim=1)

            # Converting hyps from indexes to chars
            hyp_lst = []
            for hyp, stop_ind in zip(hyps, stop_indexes):
                # in some cases the eos in not observed (e.g, for the last sentence
                # in the batch)
                if stop_ind == 0:
                    stop_ind = -1
                # Stopping when eos is observed
                hyp = hyp[0:stop_ind]
                # From index to character
                hyp_lst.append(self.label_encoder.decode_ndim(hyp))
            return predictions, hyp_lst

        return predictions


    def compute_objectives(self, predictions, batch, stage):
        """Computes the loss given the predicted and targeted outputs.

        Arguments
        ---------
        predictions : torch.tTensor
            The output tensor from `compute_forward`.
        batch : PaddedBatch
            This batch object contains all the relevant tensors for computation.
        stage : sb.Stage
            One of sb.Stage.TRAIN, sb.Stage.VALID, or sb.Stage.TEST.

        Returns
        -------
        loss : torch.Tensor
            A one-element tensor used for backpropagating the gradient.
        """
        # Reading english_encoded_chars_eos
        # Your code here. Aim for 1 line.
        enc_english_eos, english_lens = batch.english_encoded_chars_eos


        # Reading the predictions
        if stage == sb.Stage.TEST:
          predictions, hyp_lst = predictions

          for id, label, hyp in zip(batch.id, batch.english_chars, hyp_lst):
              print(id)
              print("REF: " + ''.join(label))
              print("HYP: " + ''.join(hyp))
              print('--------')

        # Compute the nnl_loss
        # Your code here. Aim for 1 line.
        loss = torch.nn.functional.nll_loss(
            predictions.transpose(1, 2), enc_english_eos,
            ignore_index=self.hparams.blank_index,
        )



        return loss


    def on_stage_end(self, stage, stage_loss, epoch):
        """Gets called at the end of an epoch.

        Arguments
        ---------
        stage : sb.Stage
            One of sb.Stage.TRAIN, sb.Stage.VALID, sb.Stage.TEST
        stage_loss : float
            The average loss for all of the data processed in this stage.
        epoch : int
            The currently-starting epoch. This is passed
            `None` during the test stage.
        """

        # Store the train loss until the validation stage.
        stage_stats = {"loss": stage_loss}
        if stage == sb.Stage.TRAIN:
            self.train_stats = stage_stats


        # Perform end-of-iteration things, like annealing, logging, etc.
        elif stage == sb.Stage.VALID:


            # Update learning rate
            old_lr, new_lr = self.hparams.lr_annealing(epoch)
            sb.nnet.schedulers.update_learning_rate(self.optimizer, new_lr)

            # The train_logger writes a summary to stdout and to the logfile.
            self.hparams.train_logger.log_stats(
                stats_meta={"epoch": epoch, "lr": old_lr},
                train_stats=self.train_stats,
                valid_stats={
                    "loss": stage_loss,
                },
            )
            # Save the current checkpoint and delete previous checkpoints.
            self.checkpointer.save_and_keep_only(
                meta={"loss": stage_stats["loss"]}, min_keys=["loss"],
            )

        # We also write statistics about test data to stdout and to the logfile.
        elif stage == sb.Stage.TEST:

            self.hparams.train_logger.log_stats(
                stats_meta={"Epoch loaded": self.hparams.epoch_counter.current},
                test_stats={
                    "loss": stage_loss,
                },
            )


def dataio_prepare(hparams):
    """This function prepares the datasets to be used in the brain class.
    It also defines the data processing pipeline through user-defined functions.


    Arguments
    ---------
    hparams : dict
        This dictionary is loaded from the `train.yaml` file, and it includes
        all the hyperparameters needed for dataset construction and loading.

    Returns
    -------
    datasets : dict
        Dictionary containing "train", "valid", and "test" keys that correspond
        to the DynamicItemDataset objects.
    """
    # Define text processing pipeline. We start from the raw text and then
    # split it into characters. The tokens with BOS are used for feeding
    # the decoder during training (right shifr), the tokens with EOS
    # are used for computing the cost function.
    @sb.utils.data_pipeline.takes("english")
    @sb.utils.data_pipeline.provides(
        "english_words",
        "english_chars",
        "english_encoded_chars_lst",
        "english_encoded_chars",
        "english_encoded_chars_eos",
        "english_encoded_chars_bos",
        )
    def ignotush_text_pipeline(english):
        """Processes the transcriptions to generate proper labels"""
        yield english
        english_chars = list(english)
        yield english_chars
        english_encoded_chars_lst = label_encoder.encode_sequence(english_chars)
        yield english_encoded_chars_lst
        english_encoded_chars = torch.LongTensor(english_encoded_chars_lst)
        yield english_encoded_chars
        english_encoded_chars_eos = torch.LongTensor(label_encoder.append_eos_index(english_encoded_chars_lst))
        yield english_encoded_chars_eos
        english_encoded_chars_bos = torch.LongTensor(label_encoder.prepend_bos_index(english_encoded_chars_lst))
        yield english_encoded_chars_bos

    @sb.utils.data_pipeline.takes("ignotush")
    @sb.utils.data_pipeline.provides("ignotush_words", "ignotush_chars", "ignotush_encoded_chars")
    def english_text_pipeline(ignotush):
        """Processes the transcriptions to generate proper labels"""
        yield ignotush
        ignotush_chars = list(ignotush)
        yield ignotush_chars
        ignotush_encoded_chars = torch.LongTensor(input_encoder.encode_sequence(ignotush_chars))
        yield ignotush_encoded_chars

    # Define datasets from json data manifest file
    # Define datasets sorted by ascending lengths for efficiency
    datasets = {}
    data_info = {
        "train": hparams["train_annotation"],
        "valid": hparams["valid_annotation"],
        "test": hparams["test_annotation"],
    }

    # The label encoder will assign a different integer to each element
    # in the output vocabulary
    input_encoder = sb.dataio.encoder.CTCTextEncoder()
    label_encoder = sb.dataio.encoder.CTCTextEncoder()

    for dataset in data_info:
        datasets[dataset] = sb.dataio.dataset.DynamicItemDataset.from_csv(
            csv_path=data_info[dataset],
            dynamic_items=[ignotush_text_pipeline, english_text_pipeline],
            output_keys=[
                "id",
                "english_words",
                "english_chars",
                "english_encoded_chars",
                "english_encoded_chars_eos",
                "english_encoded_chars_bos",
                "ignotush_words",
                "ignotush_chars",
                "ignotush_encoded_chars",
            ],
        )
        hparams[f"{dataset}_dataloader_opts"]["shuffle"] = True


    # Load or compute the label encoder
    inp_enc_file = os.path.join(hparams["save_folder"], "input_encoder.txt")

    # The blank symbol is used to indicate padding
    special_labels = {"blank_label": hparams["blank_index"]}

    input_encoder.load_or_create(
        path=inp_enc_file,
        from_didatasets=[datasets["train"]],
        output_key="ignotush_chars",
        special_labels=special_labels,
        sequence_input=True,
    )

    # Load or compute the label encoder
    lab_enc_file = os.path.join(hparams["save_folder"], "label_encoder.txt")
    special_labels = {
        "blank_label": hparams["blank_index"],
        "bos_label": hparams["bos_index"],
        "eos_label": hparams["eos_index"],
    }
    label_encoder.load_or_create(
        path=lab_enc_file,
        from_didatasets=[datasets["train"]],
        output_key="english_chars",
        special_labels=special_labels,
        sequence_input=True,
    )

    return datasets, label_encoder


if __name__ == "__main__":

    # Reading command line arguments
    hparams_file, run_opts, overrides = sb.parse_arguments(sys.argv[1:])

    # Load hyperparameters file with command-line overrides
    with open(hparams_file) as fin:
        hparams = load_hyperpyyaml(fin, overrides)

    # Create experiment directory
    sb.create_experiment_directory(
        experiment_directory=hparams["output_folder"],
        hyperparams_to_save=hparams_file,
        overrides=overrides,
    )


    # We can now directly create the datasets for training, valid, and test
    datasets, label_encoder = dataio_prepare(hparams)

    # Trainer initialization
    translate_brain = Translate(
        modules=hparams["modules"],
        opt_class=hparams["opt_class"],
        hparams=hparams,
        run_opts=run_opts,
        checkpointer=hparams["checkpointer"],
    )

    # Making label encoder accessible (needed for computer the character error rate)
    translate_brain.label_encoder = label_encoder

    # The `fit()` method iterates the training loop, calling the methods
    # necessary to update the parameters of the model. Since all objects
    # with changing state are managed by the Checkpointer, training can be
    # stopped at any point, and will be resumed on next call.
    translate_brain.fit(
        translate_brain.hparams.epoch_counter,
        datasets["train"],
        datasets["valid"],
        train_loader_kwargs=hparams["train_dataloader_opts"],
        valid_loader_kwargs=hparams["valid_dataloader_opts"],
    )

    # Load best checkpoint for evaluation
    test_stats = translate_brain.evaluate(
        test_set=datasets["test"],
        min_key="WER",
        test_loader_kwargs=hparams["test_dataloader_opts"],
    )
